In [4]:
import sys
import logging
import re
import psycopg2
import requests
import pandas as pd
from fredapi import Fred
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database

from docopt import docopt

from pprint import pprint as pp


__version__ = '0.0.1'


def get_db_engine(db_name, user, password, host, port=5432):

    db_connection = "postgresql://{user}:{password}@{host}:{port}/{database}".format(
        database=db_name,
        user=user,
        password=password,
        host=host,
        port=port
    )

    engine = create_engine(db_connection)
    if not database_exists(engine.url):
        try:
            create_database(engine.url)

        except Exception as exc:
            logger.exception('Could not create database. \nException: %r', exc)
            return exc
    return engine


def main():

    user = 'donal'
    password = 'donal'
    host = 'localhost'
    api_key = '01af77900eb060649a7c504ee0705b4d'
    db_name = 'fred_db'
    port = 5432

    engine = get_db_engine(db_name, user, password, host, port)

    fred = Fred(api_key=api_key)
    gdp = fred.get_series('GDPC1').rename('gdp')  # ('GDP')
    um_cust_sent_index = fred.get_series('UMCSENT').rename('umcsent')  # ('UM customer sentiment index')
    us_civ_unemploy_rate = fred.get_series('UNRATE').rename('unrate')  # ('US  Civilian Unemployment Rate')

    frames = [gdp, um_cust_sent_index, us_civ_unemploy_rate]
    fred_data = pd.concat(frames, axis=1)

    return fred_data

if __name__ == "__main__":
    data = main()

In [5]:
print(data)

                gdp  umcsent  unrate
1947-01-01   1934.5      NaN     NaN
1947-04-01   1932.3      NaN     NaN
1947-07-01   1930.3      NaN     NaN
1947-10-01   1960.7      NaN     NaN
1948-01-01   1989.5      NaN     3.4
1948-02-01      NaN      NaN     3.8
1948-03-01      NaN      NaN     4.0
1948-04-01   2021.9      NaN     3.9
1948-05-01      NaN      NaN     3.5
1948-06-01      NaN      NaN     3.6
1948-07-01   2033.2      NaN     3.6
1948-08-01      NaN      NaN     3.9
1948-09-01      NaN      NaN     3.8
1948-10-01   2035.3      NaN     3.7
1948-11-01      NaN      NaN     3.8
1948-12-01      NaN      NaN     4.0
1949-01-01   2007.5      NaN     4.3
1949-02-01      NaN      NaN     4.7
1949-03-01      NaN      NaN     5.0
1949-04-01   2000.8      NaN     5.3
1949-05-01      NaN      NaN     6.1
1949-06-01      NaN      NaN     6.2
1949-07-01   2022.8      NaN     6.7
1949-08-01      NaN      NaN     6.8
1949-09-01      NaN      NaN     6.6
1949-10-01   2004.7      NaN     7.9
1